In [1]:
import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
#import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string
from nltk.corpus import stopwords
import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable

In [2]:
net_emb = gsm.KeyedVectors.load_word2vec_format('user.emd')

In [36]:
user_em_df = pd.read_csv('./datasets/network.csv')

In [37]:
len(user_em_df)

895

In [38]:
user_em_df_men = user_em_df[['mention','mention_num']]
user_em_df_uname = user_em_df[['username','username_num']]

In [39]:
user_em_df_men = user_em_df_men.rename(columns={'mention':'username','mention_num':'name_num'})
user_em_df_uname = user_em_df_uname.rename(columns={'username':'username','username_num':'name_num'})

In [40]:
df_namenum = pd.concat([user_em_df_men,user_em_df_uname])
df_namenum = df_namenum.drop_duplicates().reset_index(drop=True)

In [41]:
df_namenum

,username,name_num
0,PalmerReport,418
1,bmckenz44559503,310
2,LeaBlackMiami,481
3,Catteeya,219
4,JewellE1974,641
...,...,...
723,LOALLL,676
724,jrbkjrbk,324
725,stella105eric,157
726,SamuelBriceHall,74


In [58]:
test = pd.read_csv("./datasets/test.csv") 
train = pd.read_csv("./datasets/train.csv") 

In [60]:
test = test[['username','label']]
train = train[['username','label']]

In [61]:
test = test[test['label']!=2]
train = train[train['label']!=2]

In [62]:
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)

In [64]:
train = train.drop_duplicates().reset_index(drop=True)
test = test.drop_duplicates().reset_index(drop=True)

In [68]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, stratify=train.label, test_size=0.2, random_state=42)

In [69]:
df = pd.concat([test,train,valid])
df = df.reset_index(drop=True)

In [70]:
df = df.drop_duplicates().reset_index(drop=True)

In [71]:
train = train.drop_duplicates().reset_index(drop=True)
valid = valid.drop_duplicates().reset_index(drop=True)
test = test.drop_duplicates().reset_index(drop=True)

In [75]:
df_num = pd.merge(df,df_namenum, how='outer',on='username')

In [76]:
df_num = df_num.fillna(2021)

In [77]:
df_num['name_num'] = df_num['name_num'].astype(int)

In [78]:
df_num_21 = df_num[df_num['name_num']==2021]
df_num_na21 = df_num[df_num['name_num']!=2021]

In [79]:
df_num_21 = df_num_21.reset_index(drop=True)

n = 728

for i in range(len(df_num_21)):
    df_num_21['name_num'][i] = n
    n+=1

/home/dxlab/jupyter/envs/grad_py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [80]:
df_num = pd.concat([df_num_21, df_num_na21])

In [81]:
train_data = pd.merge(train,df_num,on='username')

test_data = pd.merge(test,df_num,on='username')

valid_data = pd.merge(valid,df_num,on='username')

In [88]:
train_data.to_csv('train_net.csv',index=False)
test_data.to_csv('test_net.csv',index=False)
valid_data.to_csv('valid_net.csv',index=False)

In [24]:
labels_train = torch.tensor(train_data.label_x.values)

labels_valid = torch.tensor(valid_data.label_x.values)

labels_test = torch.tensor(test_data.label_x.values)

In [94]:
def nn_input_network(train_data):
    training_data =[]
    for i in range (0, len(train_data)):
        for j in range (0,train_data.shape[0]):
            label =  [int(train_data.label_x[j])]
            if(str(train_data['name_num'][i]) not in net_emb ):
                net_emb[str(train_data['name_num'][i])] = np.zeros(300)
            net_emb_tensor = torch.Tensor(net_emb[str(train_data['name_num'][i])])
            training_data.append(net_emb_tensor.unsqueeze(0))
            break
    return training_data

In [95]:
#####prepare training data for neural net #########
training_data_net =  nn_input_network(train_data)
#####prepare validation data for neural net #########
validation_data_net =  nn_input_network(valid_data)
#####prepare Testing data for neural net #########
testing_data_net =  nn_input_network(test_data)

/home/dxlab/jupyter/envs/grad_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  


In [97]:
# Convert the lists into tensors.
training_data_net = torch.cat(training_data_net, dim=0)
validation_data_net = torch.cat(validation_data_net, dim=0)
testing_data_net = torch.cat(testing_data_net, dim=0)

In [101]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(training_data_net, labels_train)
val_dataset = TensorDataset(validation_data_net, labels_valid)
test_dataset = TensorDataset(testing_data_net, labels_test)

In [103]:
import torch
# torch.cuda.empty_cache() 
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    #device = torch.device("cuda") # select the zeroth GPU with this line: gpu = 0
    #device = torch.cuda.set_device(1)  #wrong provide device = None  
    device = torch.device(1) #(use cuda device 1) for gpu = 1
    torch.cuda.set_device(device)
    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name())

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 3 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [104]:
!nvidia-smi

Fri Sep 17 15:18:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 440.59       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 30%   49C    P8    21W / 250W |     11MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:67:00.0 Off |                  N/A |
| 36%   58C    P8    14W / 250W |     11MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [105]:
device = torch.cuda.current_device()
device

1

In [106]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [107]:
class NetworkMLP(nn.Module):
    def __init__(self):
        super(NetworkMLP, self).__init__() 
        self.fc1 = nn.Linear(300, 150)
        
    def forward(self, X):
        #print(X.size()) # torch.Size([32, 300])
        z1 = self.fc1(X)
        #print('z1', z1, z1.size()) # torch.Size([32, 150])
        #h1 = F.relu(z1) 
        return z1 

In [108]:
class UserModel(nn.Module):
    def __init__(self, dropout):
        super().__init__()
        
        self.model_net = NetworkMLP()
        self.fc2 = nn.Linear(150, 3)
        self.dropout = nn.Dropout(dropout)
    def forward(self,  x_n): 
        prediction_net = self.model_net(x_n)
        prediction_net = self.dropout(prediction_net) # add dropout
        output = self.fc2(F.relu(prediction_net))
        return output

In [136]:
DROPOUT = 0.5

model = UserModel(DROPOUT)

In [137]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [153]:
# from transformers import AdamW

# optimizer = AdamW(model.parameters(),
#                   lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                   eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
#                 )

In [154]:
criterion = nn.CrossEntropyLoss()

In [155]:
model = model.to(device)
criterion = criterion.to(device)

In [156]:
def binary_accuracy(preds, y):
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    
    from sklearn.metrics import f1_score
    macro_f1 = f1_score(y.to("cpu"), preds.to("cpu"), average='macro')

    return acc, macro_f1

In [157]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_macro = 0
    model.train()
    for step, batch in enumerate(iterator):    
    #for batch in iterator:
        #print("batch", batch)
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)

        optimizer.zero_grad()
        net = batch[0].to(device)
        label = batch[1].to(device)
        #print("label", label, type(label),label.size()) #torch.Size([32])
        #label = label.unsqueeze(1)
        #print("label", label, type(label),label.size())
        #predictions = model(batch.text).squeeze(1)
        #predictions = model(text)
        predictions = model( net)
        #pred = model(text)
        #print("pred", pred.size()) #torch.Size([32, 3])
        #predictions = torch.argmax(pred, dim = 1)
        #print("predictions", predictions, predictions.size()) #torch.Size([32, 3])
        #loss = criterion(predictions, batch.label)
        loss = criterion(predictions, label)
        #print("loss", loss)
        #acc = binary_accuracy(predictions, batch.label)
        acc, macro_f1 = binary_accuracy(torch.argmax(predictions, dim = 1), label)
        #print("acc", acc)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_macro += macro_f1.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_macro / len(iterator)

In [158]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_macro = 0
    model.eval()
    
    with torch.no_grad():
    
        #for batch in iterator:
        for step, batch in enumerate(iterator):    
            
            net = batch[0].to(device)
            label = batch[1].to(device)

            #predictions = model(batch.text).squeeze(1)
            predictions = model( net)
            #loss = criterion(predictions, batch.label)
            loss = criterion(predictions, label)
            #acc = binary_accuracy(predictions, batch.label)
            acc, macro_f1 = binary_accuracy(torch.argmax(predictions, dim = 1), label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_macro += macro_f1.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_macro / len(iterator)

In [159]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [160]:
N_EPOCHS = 5

best_valid_loss = float('inf')
per_epoch_train_loss = []
per_epoch_val_loss = []
per_epoch_train_f1 = []
per_epoch_val_f1 = []
per_epoch_train_acc = []
per_epoch_val_acc = []
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, validation_dataloader, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    per_epoch_train_loss.append(train_loss)
    per_epoch_val_loss.append(valid_loss)
    per_epoch_train_f1.append(train_f1)
    per_epoch_val_f1.append(valid_f1)
    per_epoch_train_acc.append(train_acc)
    per_epoch_val_acc.append(valid_acc)
    
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        print("best model saved in epoch :", epoch+1 )
    torch.save(model.state_dict(), 'data/unet/net_utype'+str(epoch+1)+'.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f} | Train macro-avg-f1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f} |  Val. macro-avg-f1: {valid_f1*100:.2f}%')

best model saved in epoch : 1
Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.602 | Train Acc: 59.18 | Train macro-avg-f1: 58.45%
	 Val. Loss: 0.598 |  Val. Acc: 61.88 |  Val. macro-avg-f1: 58.50%
best model saved in epoch : 2
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.600 | Train Acc: 62.24 | Train macro-avg-f1: 61.52%
	 Val. Loss: 0.598 |  Val. Acc: 61.82 |  Val. macro-avg-f1: 58.45%
best model saved in epoch : 3
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.603 | Train Acc: 60.37 | Train macro-avg-f1: 59.48%
	 Val. Loss: 0.598 |  Val. Acc: 61.76 |  Val. macro-avg-f1: 58.40%
best model saved in epoch : 4
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.608 | Train Acc: 61.96 | Train macro-avg-f1: 60.99%
	 Val. Loss: 0.598 |  Val. Acc: 61.76 |  Val. macro-avg-f1: 58.40%
best model saved in epoch : 5
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.599 | Train Acc: 60.55 | Train macro-avg-f1: 59.72%
	 Val. Loss: 0.598 |  Val. Acc: 61.63 |  Val. macro-avg-f1: 58.29%


In [161]:
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [162]:
def evaluate_test(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_macro = 0
    model.eval()
    
    with torch.no_grad():
    
        #for batch in iterator:
        preds , true_labels = [], []
        for step, batch in enumerate(iterator):    
            text = batch[0].to(device)
            label = batch[1].to(device)

            #predictions = model(batch.text).squeeze(1)
            predictions = model(text)
            
            #loss = criterion(predictions, batch.label)
            loss = criterion(predictions, label)
            #acc = binary_accuracy(predictions, batch.label)
            acc, macro_f1 = binary_accuracy(torch.argmax(predictions, dim = 1), label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_macro += macro_f1.item()
            
            # Store predictions and true labels
            preds.append(torch.argmax(predictions, dim = 1).to('cpu').numpy())
            true_labels.append(label.to('cpu').numpy())
            
                        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_macro / len(iterator), preds, true_labels

In [163]:
from sklearn.metrics import classification_report

for epoch in range(0,5):
    model.load_state_dict(torch.load('data/unet/net_utype'+str(epoch+1)+'.pt'))
    test_loss, test_acc, test_f1, pred, label = evaluate_test(model, test_dataloader, criterion)

    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.3f} | Test macro-avg-f1: {test_f1*100:.3f}%')
    flat_predictions = np.concatenate(pred, axis=0)
    flat_labels = np.concatenate(label, axis=0)
    
    print(classification_report(flat_labels, flat_predictions, digits=4))

Test Loss: 0.621 | Test Acc: 61.699 | Test macro-avg-f1: 56.848%
              precision    recall  f1-score   support

           0     0.7722    0.3065    0.4388       199
           1     0.5647    0.9086    0.6965       197

    accuracy                         0.6061       396
   macro avg     0.6684    0.6076    0.5677       396
weighted avg     0.6689    0.6061    0.5670       396

Test Loss: 0.620 | Test Acc: 61.699 | Test macro-avg-f1: 56.848%
              precision    recall  f1-score   support

           0     0.7722    0.3065    0.4388       199
           1     0.5647    0.9086    0.6965       197

    accuracy                         0.6061       396
   macro avg     0.6684    0.6076    0.5677       396
weighted avg     0.6689    0.6061    0.5670       396

Test Loss: 0.620 | Test Acc: 61.699 | Test macro-avg-f1: 56.848%
              precision    recall  f1-score   support

           0     0.7722    0.3065    0.4388       199
           1     0.5647    0.9086    0.696